In [1]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [2]:
# Load gridded FRP data aggregated to first observation day
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_gridstats_days')
print(f"Number of unique grid days (dissolved): {grid.size().getInfo()}")
grid = grid.select(['grid_index', 'Fire_ID', 'Ig_Date', 'Last_Date', 'max_date', 'first_obs', 'last_obs'])
print(grid.first().propertyNames().getInfo())

Number of unique grid days (dissolved): 986
['system:index', 'grid_index', 'Fire_ID', 'max_date', 'first_obs', 'Ig_Date', 'Last_Date', 'last_obs']


In [3]:
# get a list of unique active fire days
fire_days = grid.aggregate_array('first_obs').getInfo()
fire_days = set(fire_days)
print(f"Number of unique 'fire days': {len(fire_days)}")

# get the min and max DOY
fire_days_doy = [datetime.strptime(day, "%Y-%m-%d").timetuple().tm_yday for day in fire_days]
doy_min = min(fire_days_doy)
doy_max = max(fire_days_doy)
print(f"Fire DOY range: {doy_min} to {doy_max}")

Number of unique 'fire days': 520
Fire DOY range: 89 to 317


In [4]:
# Load the gridmet image collection
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"\ngridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}\n")


gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']



In [5]:
# select our variables of interest:
# 1. Vapor Pressure Deficit, Energy Release Component
gridmet = gridmet.select(['vpd','erc','tmmx','vs','rmin','fm1000'])
print(gridmet.first().bandNames().getInfo())

['vpd', 'erc', 'tmmx', 'vs', 'rmin', 'fm1000']


In [8]:
def process_fire_day(ftr):
    """ 
    Calculates gridmet statistics for daily grids (dissolved)
    Including the deviation from long-term average for time-of-year
    """

    ######################################################
    # calculate the long-term average for the time-of-year
    # get the fire days (ignition, cessation)
    ig_doy = ee.Date(ftr.get('Ig_Date')).advance(-14,'days').getRelative('day', 'year')
    ls_doy = ee.Date(ftr.get('Last_Date')).advance(14,'days').getRelative('day', 'year')

    # define the 'deviation' band names
    attrs = ee.List(gridmet.first().bandNames()) # grab the band names for renaming
    attrs = attrs.map(lambda atr: ee.String(atr).cat('_dv'))
    
    # average for time-of-year of fire
    lta = gridmet.filter(
        ee.Filter.calendarRange(ig_doy, ls_doy, 'day_of_year')
    ).mean().rename(attrs)      

    ##################################
    # get daily fire dates valid range
    first_date = ee.Date(ftr.get('first_obs')) # day of first observation
    end_date = ee.Date(ftr.get('last_obs')).advance(1, 'days') # advance at least one day just in case they are the same
    
    # filter the gridmet to day of first observation
    gridmet_day = gridmet.filterDate(first_date, end_date).first()
    gridmet_anom = gridmet_day.subtract(lta).rename(attrs) # get the deviation
    image = gridmet_day.addBands(gridmet_anom)
    
    # run the reduction
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ftr.geometry().bounds().buffer(100),
        scale=4000
    )

    return ee.Feature(None, stats).set({
        'Fire_ID': ftr.get('Fire_ID'),
        'first_obs': ftr.get('first_obs'),
        'last_obs': ftr.get('last_obs')
    })

# map the function across gridcells
results = ee.FeatureCollection(grid.map(process_fire_day))
results.limit(10)

In [10]:
sample = results.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df = df[['Fire_ID', 'first_obs', 'last_obs', 'erc', 'erc_dv', 'vpd', 'vpd_dv', 'fm1000', 'fm1000_dv',
         'rmin', 'rmin_dv', 'tmmx', 'tmmx_dv', 'vs', 'vs_dv'
        ]]
df.head(10)

,Fire_ID,first_obs,last_obs,erc,erc_dv,vpd,vpd_dv,fm1000,fm1000_dv,rmin,rmin_dv,tmmx,tmmx_dv,vs,vs_dv
0,71,2017-05-21,2017-05-21,54.807692,-3.885888,0.900769,-0.077104,12.088462,1.262081,20.242308,-0.082974,291.488460,0.143907,2.592308,-1.114161
1,71,2017-05-22,2017-05-28,56.000000,-3.733707,0.880000,-0.112899,12.000000,1.398518,22.700001,2.552782,290.700012,-0.607025,3.300000,-0.472120
2,71,2017-05-24,2017-05-24,62.294430,1.974189,1.252520,0.234063,10.941114,0.459652,15.804244,-4.056106,295.654648,3.879818,3.923077,0.204069
3,71,2017-05-26,2017-05-26,65.000000,4.657209,1.220000,0.203369,10.533333,0.056103,16.300000,-3.558869,294.066671,2.352997,4.766667,1.011170
4,71,2017-05-27,2017-05-27,64.323529,4.706889,1.059706,0.065765,10.567647,-0.058743,17.638235,-2.485484,292.561759,1.146648,3.867647,0.145134
5,71,2017-05-28,2017-05-28,64.000000,4.266293,0.980000,-0.012899,10.600000,-0.001482,19.400000,-0.747219,292.200012,0.892975,2.300000,-1.472120
6,33,2017-06-15,2017-06-15,84.816327,17.516736,2.209796,0.677961,8.018368,-1.238856,4.195918,-12.665888,301.175502,3.545579,3.200000,-1.252799
7,33,2017-06-16,2017-06-16,86.164894,19.460624,2.241755,0.778281,7.883511,-1.471472,5.517553,-11.800843,300.851597,3.940556,4.767021,0.341797
8,33,2017-06-17,2017-06-18,87.000000,20.800201,2.300000,0.910609,7.600000,-1.838265,7.900000,-9.763855,301.600006,5.682922,5.100000,0.822212
9,33,2017-06-18,2017-06-18,84.000000,17.800201,1.970000,0.580609,7.600000,-1.838265,15.500000,-2.163855,301.200012,5.282928,3.800000,-0.477788


In [11]:
# Remove geometry
def remove_geometry(ftr):
    return ftr.setGeometry(None)
results = results.map(remove_geometry)
results = results.select([
    'Fire_ID','first_obs','last_obs','erc','vpd','erc_dv','vpd_dv', 'vs', 'vs_dv',
    'fm1000', 'fm1000_dv','rmin', 'rmin_dv', 'tmmx', 'tmmx_dv'
])

# Export the table. 
export_task = ee.batch.Export.table.toDrive(
    collection=results,
    description='gridstats_gridmet_full_v',
    fileNamePrefix='gridstats_gridmet_full_v',
    fileFormat='CSV', 
    folder='GRIDMET'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# # Monitor the task until it's finished
# monitor_export(export_task, 300) 

Export to Earth Engine Asset started!
